<a href="https://colab.research.google.com/github/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/img/Banner_Bootcamp.png?raw=true">

#**Projeto Final de Conclusão de Curso**
###Bootcamp Data Science Aplicada 2 by [Alura](https://www.alura.com.br/) 
####Autor: Helton Cordeiro
e-mail: heltoncordeiro@gmail.com

Junho-Agosto/2021.



---

# **Objetivo desse notebook**

Após os dados disponibilizados pelo Hospital Sírio Libânes ter sido tratados em [notebook próprio](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb) e termos trabalhando com alguns modelos de Machine Learning com seus valores default, vamos explorar os hiperparametros nesse notebook. 

Será utilizando um dicionário com o range de valores e parâmetros  que serão passados para o método **RandomizedSearchCV** utilizando a função *executa_modelos_RandomizedSearchCV* que está no arquivo funcoes.py.

A proposta da separação do [notebook de análise](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/Bootcamp_DataScience_projeto_final.ipynb) se deve ao tempo de processamento necessário para testar todos os modelos, além de facilitar futuros ajustes.

---


##O que é um hiperparâmetro?

Os hiperparâmetros contêm os dados que controlam o próprio processo de treinamento.

Seu aplicativo de treinamento lida com três categorias de dados durante o treinamento do modelo:

* Os *dados de entrada*, também chamados de dados de treinamento, formam uma coleção de registros individuais (instâncias) com as características que são importantes para o problema de machine learning. Esses dados configuram o modelo durante o treinamento para fazer predições precisas sobre novas instâncias de dados semelhantes. No entanto, os valores nos dados de entrada nunca se tornam diretamente parte do modelo.

* Os *parâmetros* do modelo são as variáveis que a técnica de machine learning escolhida usa para ajustar os dados. Por exemplo, uma rede neural profunda (DNN, na sigla em inglês) é composta por nós de processamento (neurônios), cada um com uma operação realizada nos dados enquanto eles trafegam pela rede. Quando a DNN é treinada, cada node tem um valor de peso que informa ao modelo o impacto que ele tem na predição final. Esses pesos são um exemplo dos parâmetros do modelo. De muitas formas, esses parâmetros são o modelo. Ou seja, são eles que diferenciam seu modelo específico de outros modelos do mesmo tipo que trabalham com dados semelhantes.

* Os *hiperparâmetros* são variáveis que controlam o próprio processo de treinamento. Por exemplo, faz parte da configuração de uma rede neural profunda decidir quantas camadas ocultas de nós precisam ser usadas entre a camada de entrada e a camada de saída, bem como quantos nós cada camada precisa usar. Essas variáveis não estão diretamente relacionadas aos dados de treinamento. Elas são variáveis de configuração. Os parâmetros mudam durante um job de treinamento, enquanto os hiperparâmetros geralmente permanecem constantes durante um job.


Os parâmetros do modelo são otimizados (ou seja, "ajustados") pelo processo de treinamento. Você executa os dados por meio das operações do modelo, compara a predição resultante com o valor real de cada instância de dados, avalia a precisão e ajusta até encontrar os melhores valores. Os hiperparâmetros são ajustados por meio da execução de todo o job de treinamento, a observação da precisão agregada e o ajuste. Nos dois casos, você está modificando a composição do modelo tentando encontrar a melhor combinação para lidar com o problema.

Fonte: [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)


##Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random
from scipy.stats import randint

import warnings
warnings.simplefilter(action='ignore')

## Importando as funções auxiliares

* Arquivo funcoes.py contém as funções python que serão utilizados pelo projeto.

In [2]:
from funcoes import executa_modelos_RandomizedSearchCV

## Carregando os dados
Vamos carregar o arquivo de dados tratados e sem as colunas com alta correção.

O processo de tratamento dos dados estão nesse notebook [projeto_final_tratamento_dados.ipynb](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb).


### Realizando a carga de dados

In [3]:
url='https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/dados/Kaggle_Sirio_Libanes_ICU_Prediction-tratado-sem-corr.xls?raw=true'
dados_raw = pd.read_excel(url, index_col=0)
dados = dados_raw.copy()



---


#Modelos de Machine Learning
Relação dos Modelos que serão testados nesse projeto, foram selecionados por resolverem problemas de classificação binária e aprendizado supervisionado.


1. [KNeighborsClassifier](
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

2. [SVC - Linear](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

3. [SVC - RBF](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

4. [GaussianProcessClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html)

5. [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

6. [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

7. [Neural Net (MLP)](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

8. [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

9. [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

10. [ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)

11. [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)


---



##Modelos

Criando as instâncias dos modelos para processamento.

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

names = ["KNeighbors", "LinearSVC", "RBFSVC", "Gaussian",
         "DecisionTree", "RandomForest", "NeuralMLP", "AdaBoost",
         "Gradient", "ExtraTrees", "LogisticRegression"]

classes = [
    KNeighborsClassifier(),
    SVC(kernel="linear", probability=True),
    SVC(kernel="rbf", probability=True),
    GaussianProcessClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    ExtraTreesClassifier(),
    LogisticRegression(max_iter=2000) ]

##Testando Hiperparâmetros

Definindo o dicionário de parâmetros a serem testados.

In [7]:
hiperparams = { 
    "KNeighbors"  : { 
        "n_neighbors" : randint(2, 20),
        "weights" : ["uniform", "distance"],
        "leaf_size" : randint(25, 100),
        "metric" : ["minkowski","wminkowski", "euclidean"] 
    },
    "LinearSVC"   : { 
        "gamma" : ["scale","auto"],
        "C": randint(0,2)
    },
    "RBFSVC"      : { 
        "gamma" : ["scale","auto"],
        "C": randint(0,2)
    },
    "Gaussian"    : {
        "n_restarts_optimizer" : randint(0, 5),
        "max_iter_predict" : randint(50,500),
        "warm_start" : [False, True] 
    },
    "DecisionTree": {
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(2, 10),
        "min_samples_split" : randint(2, 129),
        "min_samples_leaf" : randint(1, 129)
    },
    "RandomForest": {
        "n_estimators" :randint(50, 400),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(3, 100),
        "min_samples_split" : randint(2, 100),
        "min_samples_leaf" : randint(1, 100),
        "bootstrap" : [True, False],
        "warm_start" : [False, True]
    },
    "NeuralMLP"   : {
        "activation" : ["identity", "logistic", "tanh", "relu"],
        "solver" : ["lbfgs", "sgd", "adam"],
        "shuffle" : [True, False],
        "warm_start" : [False, True]
    },
    "AdaBoost"    : {
        "n_estimators" : randint(25, 200),
        "algorithm" : ["SAMME","SAMME.R"]
    },
    "Gradient"    : {
        "n_estimators" : randint(50,500), 
        "min_samples_split" : randint(2, 25),
        "min_samples_leaf" : randint(1, 10), 
        "max_depth" : randint(1,8), 
        "warm_start" : [False, True]
    },
    "ExtraTrees"  : {
        "n_estimators" :randint(50, 500),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(0, 10),
        "min_samples_split" : randint(2, 129),
        "min_samples_leaf" : randint(1, 129),
        "bootstrap" : [True, False],
        "warm_start" : [False, True] 
    },
    "LogisticRegression" : {
        "solver"   : ["newton-cg", "lbfgs", "liblinear", "sag", "saga"], 
        "max_iter" : randint(1500,10000),
        "warm_start" : [False, True] 
    }
}

##Vamos processar as modelos utilizando 5 splits de dados, com 10 repetições e 15 iterações.

[RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

In [8]:
dfmodelosRand = executa_modelos_RandomizedSearchCV(names, classes, dados, 5, 10, hiperparams, 15)

Modelo: KNeighbors 	 tempo: 7 segundos
Modelo: LinearSVC 	 tempo: 14 segundos
Modelo: RBFSVC 	 tempo: 17 segundos
Modelo: Gaussian 	 tempo: 58 segundos
Modelo: DecisionTree 	 tempo: 5 segundos
Modelo: RandomForest 	 tempo: 195 segundos
Modelo: NeuralMLP 	 tempo: 338 segundos
Modelo: AdaBoost 	 tempo: 203 segundos
Modelo: Gradient 	 tempo: 337 segundos
Modelo: ExtraTrees 	 tempo: 235 segundos
Modelo: LogisticRegression 	 tempo: 55 segundos


In [9]:
dfmodelosRand

,Modelo,AUC,Train AUC,Std AUC,Best Params,Tempo
Nome,,,,,,
RandomForest,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.785994,0.909343,0.059556,"{'bootstrap': True, 'criterion': 'gini', 'max_...",195
Gradient,"([DecisionTreeRegressor(ccp_alpha=0.0, criteri...",0.752666,0.997117,0.064610,"{'max_depth': 2, 'min_samples_leaf': 7, 'min_s...",337
LogisticRegression,"LogisticRegression(C=1.0, class_weight=None, d...",0.751311,0.844124,0.058534,"{'max_iter': 4947, 'solver': 'lbfgs', 'warm_st...",55
NeuralMLP,"MLPClassifier(activation='logistic', alpha=0.0...",0.748788,0.812865,0.058607,"{'warm_start': True, 'solver': 'adam', 'shuffl...",338
ExtraTrees,"(ExtraTreeClassifier(ccp_alpha=0.0, class_weig...",0.745225,0.818170,0.066434,"{'bootstrap': False, 'criterion': 'entropy', '...",235
LinearSVC,"SVC(C=1, break_ties=False, cache_size=200, cla...",0.737272,0.849511,0.062095,"{'C': 1, 'gamma': 'auto'}",14
DecisionTree,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...",0.731548,0.842706,0.059752,"{'criterion': 'gini', 'max_depth': 8, 'min_sam...",5
RBFSVC,"SVC(C=1, break_ties=False, cache_size=200, cla...",0.723447,0.776156,0.062797,"{'C': 1, 'gamma': 'scale'}",17
Gaussian,"GaussianProcessClassifier(copy_X_train=True, k...",0.716824,0.950327,0.069887,"{'max_iter_predict': 296, 'n_restarts_optimize...",58


In [10]:
dfmodelosRand[['AUC','Train AUC', 'Std AUC', 'Tempo']]

,AUC,Train AUC,Std AUC,Tempo
Nome,,,,
RandomForest,0.785994,0.909343,0.059556,195
Gradient,0.752666,0.997117,0.064610,337
LogisticRegression,0.751311,0.844124,0.058534,55
NeuralMLP,0.748788,0.812865,0.058607,338
ExtraTrees,0.745225,0.818170,0.066434,235
LinearSVC,0.737272,0.849511,0.062095,14
DecisionTree,0.731548,0.842706,0.059752,5
RBFSVC,0.723447,0.776156,0.062797,17
Gaussian,0.716824,0.950327,0.069887,58


# Lista dos melhores parâmetros

Após aproximadamente 25 minutos de processamentos temos a lista dos melhores modelos e parâmetros de acordo com as configurações apresentadas no dicionário ```hiperparams```.

In [11]:
for indice in names:
   display(indice, dfmodelosRand.at[indice,'Best Params'])

'KNeighbors'

{'leaf_size': 46,
 'metric': 'minkowski',
 'n_neighbors': 17,
 'weights': 'distance'}

'LinearSVC'

{'C': 1, 'gamma': 'auto'}

'RBFSVC'

...

{'max_depth': 2,
 'min_samples_leaf': 7,
 'min_samples_split': 10,
 'n_estimators': 100,
 'warm_start': False}

'ExtraTrees'

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 7,
 'min_samples_leaf': 13,
 'min_samples_split': 102,
 'n_estimators': 188,
 'warm_start': True}

'LogisticRegression'

{'max_iter': 4947, 'solver': 'lbfgs', 'warm_start': False}

In [12]:
for indice in names:
   display(indice, dfmodelosRand.at[indice,'Modelo'])

'KNeighbors'

KNeighborsClassifier(algorithm='auto', leaf_size=46, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=17, p=2,
                     weights='distance')

'LinearSVC'

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

'RBFSVC'

...

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=2,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=7, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

'ExtraTrees'

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=7, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=13, min_samples_split=102,
                     min_weight_fraction_leaf=0.0, n_estimators=188,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=True)

'LogisticRegression'

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4947,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)



---


## Referências

* [Kaggle - COVID-19 - Clinical Data to assess diagnosis - Sírio Libanês](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)
* [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
* [SciKit Learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Configurar o treinamento do AutoML com Python](https://docs.microsoft.com/pt-br/azure/machine-learning/how-to-configure-auto-train)
* [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)
